In [1]:
import pickle
import numpy as np
import pandas as pd

In [2]:
Asset_PATH = "/home/zhimin90/DePaul/CSC695/Github/CSC695-Masters-Research-on-Neural-Network-Time-Series-Attention-Models/Asset/"

#### Load Gaussian KDE timeseries

In [3]:
density_matrix_t_series = pickle.load(open(Asset_PATH  + 'density_matrix_t_series_500x500.pickle', 'rb'))

In [4]:
density_matrix_t_series.shape

(198, 500, 500)

In [5]:
print(np.max(density_matrix_t_series))
print(np.min(density_matrix_t_series))

1376.5672531534574
-1.1156221503685051e-13


In [6]:
density_matrix_t_series.shape[0]

198

In [7]:
train, valid = (60,20)
test = 20

total_len = density_matrix_t_series.shape[0]
train_index = range(0,int(train/100*total_len))

In [8]:
valid_index = range(train_index[-1]+1,train_index[-1] + int(valid/100*total_len))
test_index = range(valid_index[-1]+1,valid_index[-1] + int(test/100*total_len))

In [9]:
flattened_matrix_np = np.reshape(density_matrix_t_series, 
                                 (density_matrix_t_series.shape[0],
                                  density_matrix_t_series.shape[1]*density_matrix_t_series.shape[2]))
flattened_matrix_np.shape

(198, 250000)

In [44]:
np.max(flattened_matrix_np[train_index])

1304.4451089207516

In [85]:
scaler_max = np.max(flattened_matrix_np[train_index])
scaler_min = np.min(flattened_matrix_np[train_index])

X_train = (flattened_matrix_np[train_index] - scaler_min)/(scaler_max - scaler_min)


In [86]:
X_train = X_train.reshape(X_train.shape[0],
                          1,    
                          X_train.shape[1])
X_train.shape

(118, 1, 250000)

In [87]:
X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],int(X_train.shape[2]**(1/2)),int(X_train.shape[2]**(1/2)),1)
X_train.shape

(118, 1, 500, 500, 1)

In [88]:
print(np.max(X_train))
print(np.min(X_train))

1.0
0.0


#### We transform the whole train, valid, test based on scaler fitted on train

In [89]:
np.max(flattened_matrix_np)

1376.5672531534574

In [90]:
flattened_matrix_np.dtype

dtype('float64')

In [91]:
scaled_flattened_matrix = (flattened_matrix_np - scaler_min)/(scaler_max - scaler_min)

In [92]:
flattened_matrix_np.shape

(198, 250000)

In [93]:
scaled_flattened_matrix.shape

(198, 250000)

In [94]:
np.where(scaled_flattened_matrix > 10)

(array([], dtype=int64), array([], dtype=int64))

In [95]:
flattened_matrix_np[118,33433]

0.0003376003735987424

In [96]:
scaled_flattened_matrix[118,33433]

2.5880765039597745e-07

In [98]:
print(np.max(scaled_flattened_matrix))
print(np.min(scaled_flattened_matrix))

1.0552895202254826
0.0


#### We reshape to (sample, timestep, inputshape)

In [99]:
scaled_flattened_matrix = scaled_flattened_matrix.reshape(scaled_flattened_matrix.shape[0],
                                                           1,
                                                           scaled_flattened_matrix.shape[1])
scaled_flattened_matrix.shape

(198, 1, 250000)

#### We set the target to be 7 days aggregated KDE or 1 single frame forward

In [100]:
def reshape(X_set):
    if len(X_set.shape)>2:
        return X_set.reshape(X_set.shape[0],
                             X_set.shape[1],
                             int(X_set.shape[2]**(1/2)),int(X_set.shape[2]**(1/2)),1)
    else:
        return X_set.reshape(X_set.shape[0],int(X_set.shape[1]**(1/2)),int(X_set.shape[1]**(1/2)),1)

In [101]:
Y_train = scaled_flattened_matrix[np.array(train_index)+1].reshape(
    scaled_flattened_matrix[np.array(train_index)+1].shape[0],scaled_flattened_matrix[np.array(train_index)+1].shape[2])
#Y_train = reshape(Y_train)

X_valid = scaled_flattened_matrix[np.array(valid_index)]
X_valid = reshape(X_valid)

Y_valid = scaled_flattened_matrix[np.array(valid_index)+1].reshape(
    scaled_flattened_matrix[np.array(valid_index)+1].shape[0],scaled_flattened_matrix[np.array(valid_index)+1].shape[2])
#Y_valid = reshape(Y_valid)

X_test = scaled_flattened_matrix[np.array(test_index)]
X_test = reshape(X_test)

Y_test = scaled_flattened_matrix[np.array(test_index)+1].reshape(
    scaled_flattened_matrix[np.array(test_index)+1].shape[0],scaled_flattened_matrix[np.array(test_index)+1].shape[2])
#Y_test = reshape(Y_test)

In [102]:
print(X_train[0:2])
print('-'*100)
print(Y_train[0])

[[[[[8.12321585e-17]
    [8.09930489e-17]
    [8.06092709e-17]
    ...
    [8.34700899e-17]
    [8.40057474e-17]
    [8.43639394e-17]]

   [[8.06441777e-17]
    [8.14045982e-17]
    [8.20716042e-17]
    ...
    [8.32796997e-17]
    [8.34750980e-17]
    [8.45655654e-17]]

   [[7.83087652e-17]
    [7.82554211e-17]
    [7.99920988e-17]
    ...
    [7.93122549e-17]
    [7.96093825e-17]
    [8.03617882e-17]]

   ...

   [[8.11629885e-17]
    [8.07265227e-17]
    [8.10887389e-17]
    ...
    [8.30332811e-17]
    [8.33472118e-17]
    [8.39991990e-17]]

   [[8.43633253e-17]
    [8.37326263e-17]
    [8.33673748e-17]
    ...
    [8.24431394e-17]
    [8.27789793e-17]
    [8.31071626e-17]]

   [[8.64787219e-17]
    [8.58242011e-17]
    [8.54787490e-17]
    ...
    [8.52050047e-17]
    [8.57013970e-17]
    [8.58645788e-17]]]]



 [[[[8.81826796e-17]
    [8.75643052e-17]
    [8.65007782e-17]
    ...
    [9.06523126e-17]
    [9.08102185e-17]
    [9.03362507e-17]]

   [[8.53153084e-17]
    [8.62142792

In [103]:
print(X_valid[0:2])
print('-'*100)
print(Y_valid[0])

[[[[[8.70214962e-17]
    [8.73210668e-17]
    [8.80642341e-17]
    ...
    [8.73355447e-17]
    [8.77599661e-17]
    [8.70777175e-17]]

   [[8.28472269e-17]
    [8.37461790e-17]
    [8.50141885e-17]
    ...
    [8.37419806e-17]
    [8.48074214e-17]
    [8.51124933e-17]]

   [[8.20031405e-17]
    [8.18877837e-17]
    [8.20265656e-17]
    ...
    [8.49793227e-17]
    [8.51233676e-17]
    [8.56937428e-17]]

   ...

   [[8.15679661e-17]
    [8.20359397e-17]
    [8.32880351e-17]
    ...
    [8.98107137e-17]
    [8.91339567e-17]
    [8.77974708e-17]]

   [[8.43729932e-17]
    [8.46887062e-17]
    [8.52713423e-17]
    ...
    [8.55857815e-17]
    [8.57208775e-17]
    [8.53765063e-17]]

   [[8.77184968e-17]
    [8.79419103e-17]
    [8.65964991e-17]
    ...
    [8.84076051e-17]
    [9.02798182e-17]
    [9.05385656e-17]]]]



 [[[[8.36619199e-17]
    [8.20291365e-17]
    [8.21814850e-17]
    ...
    [8.25357214e-17]
    [8.38515872e-17]
    [8.55404971e-17]]

   [[8.36819915e-17]
    [8.39861755

In [104]:
print(X_train.shape)
print(Y_train.shape)
print(X_valid.shape)
print(Y_valid.shape)
print(X_test.shape)
print(Y_test.shape)

(118, 1, 500, 500, 1)
(118, 250000)
(38, 1, 500, 500, 1)
(38, 250000)
(38, 1, 500, 500, 1)
(38, 250000)


In [105]:
import os, datetime
import tensorflow as tf
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Bidirectional, GaussianNoise, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Concatenate, Input, LSTM, RepeatVector, TimeDistributed
import tensorflow.keras.layers as layers

In [106]:
checkpoint_filepath = Asset_PATH+'checkpoints/'

## 2D CNN LSTM model

In [109]:
serie_size, time_step, n_features_row, n_features_column, no_channel = X_train.shape

epochs = 100
batch = 48
lr = .001
loss='mae'
METRIC_ACCURACY = 'mae'

model = Sequential()

'''
model.add(
    TimeDistributed(
        Conv2D(10, (2,2), activation='relu', 
               padding='same', 
               input_shape=(n_features_row, n_features_column, no_channel)),
               input_shape=(time_step,n_features_row, n_features_column, no_channel)))

model.add(
    TimeDistributed(
        MaxPooling2D(pool_size=(2, 2))))
'''

model.add(TimeDistributed(Flatten(),input_shape=(time_step,n_features_row, n_features_column, no_channel)))

model.add(LSTM(50))
model.add(Dense(100, kernel_initializer='glorot_normal', activation='relu'))
model.add(Dense(Y_train.shape[1]))

model.summary()

adam = optimizers.Adam(lr)
model.compile(loss='mean_absolute_error',
                   metrics=['mae'], 
                   optimizer=adam)


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_2 (TimeDist (None, 1, 250000)         0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 50)                50010200  
_________________________________________________________________
dense_4 (Dense)              (None, 100)               5100      
_________________________________________________________________
dense_5 (Dense)              (None, 250000)            25250000  
Total params: 75,265,300
Trainable params: 75,265,300
Non-trainable params: 0
_________________________________________________________________


In [110]:
EPOCHS = 5

earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor="val_loss",
    min_delta=0,
    patience=EPOCHS*10,
    verbose=1,
    mode="auto",
    baseline=None,
    restore_best_weights=True,)

logdir = os.path.join("./logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_cbk = tf.keras.callbacks.TensorBoard(log_dir=logdir)

modelfilename = checkpoint_filepath + '_'+\
                    datetime.datetime.now().strftime("%Y%m%d-%H%M%S")\
                    + "_model.hdf5"

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    modelfilename,
    save_weights_only=False,
    monitor='val_mae',#'val_accuracy',
    mode='min',
    verbose=1,
    period = 1,
    save_best_only=True)



model_history = model.fit(X_train,Y_train, 
                             validation_data=(X_valid, Y_valid),
                             epochs=20, 
                             batch_size=20, 
                             #callbacks=[tensorboard_cbk,
                             #hp.KerasCallback(logdir, hparams),
                             #earlystop_callback,
                             #model_checkpoint_callback]
                                   )

Train on 118 samples, validate on 38 samples
Epoch 1/20
118/118 [==============================] - 2s 13ms/sample - loss: 0.0035 - mae: 0.0035 - val_loss: 0.0030 - val_mae: 0.0030
Epoch 2/20
118/118 [==============================] - 0s 4ms/sample - loss: 0.0027 - mae: 0.0027 - val_loss: 0.0023 - val_mae: 0.0023
Epoch 3/20
118/118 [==============================] - 0s 3ms/sample - loss: 0.0021 - mae: 0.0021 - val_loss: 0.0019 - val_mae: 0.0019
Epoch 4/20
118/118 [==============================] - 0s 4ms/sample - loss: 0.0018 - mae: 0.0018 - val_loss: 0.0017 - val_mae: 0.0017
Epoch 5/20
118/118 [==============================] - 0s 4ms/sample - loss: 0.0016 - mae: 0.0016 - val_loss: 0.0016 - val_mae: 0.0016
Epoch 6/20
118/118 [==============================] - 0s 4ms/sample - loss: 0.0015 - mae: 0.0015 - val_loss: 0.0015 - val_mae: 0.0015
Epoch 7/20
118/118 [==============================] - 0s 4ms/sample - loss: 0.0014 - mae: 0.0014 - val_loss: 0.0015 - val_mae: 0.0015
Epoch 8/20
118/1

In [111]:
valid_loss, valid_acc = model.evaluate(X_valid, Y_valid, verbose=2) # 5/2020 nt: use validation set
print ("valid_accuracy_MAE=%s, valid_loss=%s" % (valid_acc, valid_loss))

38/38 - 0s - loss: 0.0013 - mae: 0.0013
valid_accuracy_MAE=0.0013199833, valid_loss=0.0013199832327173728


In [112]:
Predict = model.predict(X_valid)

In [113]:
X_valid[0]

array([[[[8.70214962e-17],
         [8.73210668e-17],
         [8.80642341e-17],
         ...,
         [8.73355447e-17],
         [8.77599661e-17],
         [8.70777175e-17]],

        [[8.28472269e-17],
         [8.37461790e-17],
         [8.50141885e-17],
         ...,
         [8.37419806e-17],
         [8.48074214e-17],
         [8.51124933e-17]],

        [[8.20031405e-17],
         [8.18877837e-17],
         [8.20265656e-17],
         ...,
         [8.49793227e-17],
         [8.51233676e-17],
         [8.56937428e-17]],

        ...,

        [[8.15679661e-17],
         [8.20359397e-17],
         [8.32880351e-17],
         ...,
         [8.98107137e-17],
         [8.91339567e-17],
         [8.77974708e-17]],

        [[8.43729932e-17],
         [8.46887062e-17],
         [8.52713423e-17],
         ...,
         [8.55857815e-17],
         [8.57208775e-17],
         [8.53765063e-17]],

        [[8.77184968e-17],
         [8.79419103e-17],
         [8.65964991e-17],
         ...,
 

In [114]:
X_valid[1]

array([[[[8.36619199e-17],
         [8.20291365e-17],
         [8.21814850e-17],
         ...,
         [8.25357214e-17],
         [8.38515872e-17],
         [8.55404971e-17]],

        [[8.36819915e-17],
         [8.39861755e-17],
         [8.43657916e-17],
         ...,
         [8.10399283e-17],
         [8.25418081e-17],
         [8.36688559e-17]],

        [[8.64167425e-17],
         [8.63895574e-17],
         [8.65183024e-17],
         ...,
         [8.44953784e-17],
         [8.43569672e-17],
         [8.48454075e-17]],

        ...,

        [[8.36364861e-17],
         [8.39945517e-17],
         [8.30546468e-17],
         ...,
         [7.88067427e-17],
         [8.01550658e-17],
         [8.15357599e-17]],

        [[8.58994855e-17],
         [8.74110402e-17],
         [8.76733713e-17],
         ...,
         [8.48234064e-17],
         [8.56931321e-17],
         [8.63135742e-17]],

        [[8.32451470e-17],
         [8.34385776e-17],
         [8.22816155e-17],
         ...,
 

In [115]:
Predict.shape

(38, 250000)

In [116]:
print(Predict[1])
print(Y_valid[0])

[ 9.9498677e-05 -1.6454863e-04 -2.2687600e-04 ... -4.4319814e-04
  6.1328174e-05  6.1887491e-05]
[8.36619199e-17 8.20291365e-17 8.21814850e-17 ... 8.47975140e-17
 8.58118597e-17 8.70728452e-17]


In [117]:
(Predict[0]-Y_valid[0]).shape

(250000,)

In [118]:
max(Predict[0])

0.14102244

In [119]:
min(Predict[0])

-0.0007073486

In [120]:
np.mean(abs(Predict[0]-Y_valid[0]))

0.0013158553258330286

In [121]:
np.where(np.abs(Predict[0]-Y_valid[0])>1000000)

(array([], dtype=int64),)

In [122]:
np.abs(Predict[0]-Y_valid[0])

array([1.03387429e-04, 1.64814119e-04, 2.25428143e-04, ...,
       4.48387233e-04, 5.83678484e-05, 5.85451926e-05])

In [123]:
np.max(Y_valid)

1.0552895202254826

In [124]:
Y_valid[0][35428]

0.00011184508114981453

In [125]:
Predict[0][35428]

-6.137701e-05

In [126]:
(Predict[0]-Y_valid[0])[35428]

-0.00017322209099235932

In [127]:
np.abs(Predict[0]-Y_valid[0])[33433]

0.0001626372873387962